# Project 02 - Wrangle and Analyze Data

##### Student Tags

Author: Anderson Hitoshi Uyekita    
Project: Wrangle and Analyze Data  
Course: Data Science - Foundation II  
COD: ND111  
Date: 26/12/2018    
Version: 1.0

***

## Table of Contents
<ul>
<li><a href="#intro">1. Introduction</a></li>
<li><a href="#gathering">2. Data Gathering</a></li>
<li><a href="#assesing">3. Data Assessing</a></li>
<li><a href="#cleaning">4. Data Cleaning</a></li>
<li><a href="#appendix">Appendix</a></li>
<li><a href="#references">References</a></li>
</ul>

***

## Synopsis


***

## 1. Introduction  <a id='intro'></a>

This Jupyter Notebook aims to document the process of Gathering, Assessing, and Cleaning of Data Science Foundations II Nanodegree Program Project 02.


### 1.1. Reproducibility

I have written this report using the Jupyter Notebook, which will allow anyone to reproduce this document in a local computer. I have made a great effort to document every step to turn this document understabable. 

#### Work envinronment

I have performed this project using:

* Dell Notebook Inspiron 7348;
* Intel(R) Core(TM) i7-5500U CPU @ 2.40GHz 2.40GHz;
* 8.00 GB, and;
* Windows 10 Pro 64-bits.

#### Softwares

There are few requirements to reproduce this project, which could be regular web Browser. I have written the Jupyter Notebook using Opera.

* Python (version 3.7.0);
* Jupyter Notebook (version 5.6.0);
* Opera (version 57.0.3098.106), and;
* Atom (version 1.33.1 ia32).

I have used the Atom to push to Github repository and to visualize some files, and nothing more than this.

#### Packages

I kindly ask you to install each of this packages before you run the next steps.

* Pandas (version 0.23.4);
* requests (version 2.20.1), and;
* tweepy (version 3.7.0).

#### Repository

You can access all files of this report in this repository:

* https://github.com/AndersonUyekita/ND111_data_science_foundations_02


### 1.2. Importing Libraries

For this project I will use some libraries to manage data frames, download file, Twitter API Client code, etc.. Let's import this packages to the Jupyter Notebook environment.

In [1]:
# Importing Library.
import pandas as pd

# Importing OS package to write and open files.
import os

# Importing the requests to create a object from a given URL.
import requests

# Importing the Client Code to use the Twitter API.
import tweepy

# Importing the json package to easy my task with json files
import json

***

## 2. Data Gathering <a id='gathering'></a>

There are two (2) files hosted in Udacity website that I must have to gather.

* `image-predictions-3.tsv`, and;
* `twitter-archive-enhanced-2.csv`.

Later, I will access the Twitter API to gather additional data.

### 2.2. Downloading Files from Udacity Website

First, I will create a folder to store the data and later I will download each file.

In [2]:
# Creating a folder to store the files.
folder = "01-Dataset" # Folder's name.

# Checking if the folder already exist.
if not folder in os.listdir():
    os.mkdir(folder)  # Creating the folder when 01-Dataset do not exist.

# List of files.
list_url = ['https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5bf60c1e_twitter-archive-enhanced-2/twitter-archive-enhanced-2.csv',
            'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv',
            'https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5bf60c69_image-predictions-3/image-predictions-3.tsv',
            'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv']

# Loop to download the three files hosted in Udacity website.
for url in list_url:
    if not url.split('/')[-1] in os.listdir(folder):                              # Check if the files already exist.
        response = requests.get(url)                                                  # Creating an object of the given URL.
        with open(os.path.join(folder,url.split('/')[-1]), mode = 'wb') as file: # Defining the file name.
            file.write(response.content)                                          # Saving the object.

_Obs.: As you can see, I have downloaded 4 (four) files because along the instruction I have found some link relating to the same document, but with different paths. I decided to download all files to compare the differences between each one, and fortunatelly the duplicated files are equals._


#### 2.2.1. Loading the files

Now that all file are in the local machine, let's load each file into the Jupyter Notebook environment.

In [3]:
# Loading the image precictions
df_img = pd.read_csv('01-Dataset/image-predictions.tsv', sep = '\t') # tsv file need to configure the sep to tabular.

# Loading the archive of WeRateDogs.
df_ach = pd.read_csv('01-Dataset/twitter-archive-enhanced-2.csv')

These three files are ready to the next step of Data Wrangling.

### 2.3. Twitter API

One of the Project requirements is to access the Twitter API to create the `tweet_json.txt`. I will use the `tweepy` package client code to access the Twitter API.

#### Sensible Data

Please, fill with your own `consumer_key`, `consumer_secret`, `access_token` and `access_secret`.

In [4]:
# Complete with your own keys and secretes.
consumer_key = ''                               # API key
consumer_secret = ''   # API secret key
access_token = ''      # Access token
access_secret = ''          # Access token secret

#### OAuth Authentication

Based on the given `consumer_key`, `consumer_secret`, `access_token` and `access_secret` I will access the Twitter API.

In [5]:
# Authentication (default configuration).
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

Based on the [tweepy documentation][tweepy_wb], there are some ways to gather the data from the Twitter.

I want all the twittes of `@dog_rates` account, and to do so I will use the [Cursor][tweepy_cursor] constructor. I have notice that Twitter has increased the number of characters from 140 to 280, and to display it properly I must set a configuration called `tweet_mode` to `extended`.

I will create an auxiliary dictionary called `twt_json`, later on I will convert this dictionary to a DataFrame.

[tweepy_wb]: https://tweepy.readthedocs.io/en/3.7.0/getting_started.html
[tweepy_cursor]: https://tweepy.readthedocs.io/en/3.7.0/cursor_tutorial.html

In [6]:
# Initializing the tweet_json.txt columns.
twt_json = {'id':[],
            'created_at':[],
            'in_reply_to_status_id':[],
            'in_reply_to_user_id':[],
            'retweet_count':[],
            'favorite_count':[],
            'source':[],
            'text':[],
            'expanded_urls':[],
            'retweeted_status_id':[],
            'retweeted_status_user_id':[],
            'retweeted_status_timestamp':[]}

The constructor `tweepy.Cursor(api.user_timeline, id = '@dog_rates', tweet_mode = 'extended')` will generate a JSON file, which is almost a dictionary. Before gathering any data, I need to understand what are the `key`'s of this dictionary/JSON file, because I will need it to find any data.

I have created an [appendix][] with all details of the JSON file, and based on this appendix, I have decided which variable I will gather.

* _json;
    * id;
    * created_at;
    * in_reply_to_status_id;
    * in_reply_to_user_id;
    * retweet_count;
    * favorite_count;
    * source;
    * full_text;
    * expanded_urls;
    * retweeted_status_id;
    * retweeted_status_user_id, and;
    * retweeted_status_timestamp.

Keep in mind, there are dictionaries nested, for this reason, I have had to index twice or more times, as you can see in the `media_url` and `retweeted_status_timestamp` cases.

In [7]:
# Initializing the twt_json columns.
twt_json = {'id':[],
            'created_at':[],
            'in_reply_to_status_id':[],
            'in_reply_to_user_id':[],
            'retweet_count':[],
            'favorite_count':[],
            'source':[],
            'full_text':[],
            'expanded_urls':[],
            'retweeted_status_id':[],
            'retweeted_status_user_id':[],
            'retweeted_status_timestamp':[]}


def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
        except StopIteration:
            break


# Loop to read line by line (here called status) the constructor.
for status in limit_handled(tweepy.Cursor(api.user_timeline, id = '@dog_rates', tweet_mode = 'extended').items()):
    try:
        twt_json['expanded_urls'].append(status._json['entities']['media'][0]['media_url'])
    except:
        twt_json['expanded_urls'].append('')
    try:
        twt_json['retweeted_status_timestamp'].append(status._json['retweeted_status']['created_at'])
    except:
        twt_json['retweeted_status_timestamp'].append('')
    try:
        twt_json['retweeted_status_id'].append(status._json['retweeted_status']['id'])
    except:
        twt_json['retweeted_status_id'].append('')
    try:
        twt_json['retweeted_status_user_id'].append(status._json['retweeted_status']['entities']['user_mentions'][0]['id'])
    except:
        twt_json['retweeted_status_user_id'].append('')
    twt_json['id'].append(status._json['id'])
    twt_json['created_at'].append(status._json['created_at'])
    twt_json['in_reply_to_status_id'].append(status._json['in_reply_to_status_id'])
    twt_json['in_reply_to_user_id'].append(status._json['in_reply_to_user_id'])
    twt_json['retweet_count'].append(status._json['retweet_count'])
    twt_json['favorite_count'].append(status._json['favorite_count'])
    twt_json['source'].append(status._json['source'])
    twt_json['full_text'].append(status._json['full_text'])
    print(status._json['created_at'])

Wed Dec 26 19:18:12 +0000 2018
Wed Dec 26 19:17:59 +0000 2018
Wed Dec 26 18:04:20 +0000 2018
Wed Dec 26 17:26:55 +0000 2018
Wed Dec 26 17:16:22 +0000 2018
Wed Dec 26 03:15:56 +0000 2018
Wed Dec 26 02:25:42 +0000 2018
Wed Dec 26 02:24:09 +0000 2018
Wed Dec 26 02:23:16 +0000 2018
Wed Dec 26 01:39:54 +0000 2018
Wed Dec 26 01:27:12 +0000 2018
Wed Dec 26 00:40:00 +0000 2018
Wed Dec 26 00:35:08 +0000 2018
Wed Dec 26 00:23:18 +0000 2018
Tue Dec 25 23:15:32 +0000 2018
Tue Dec 25 22:27:00 +0000 2018
Tue Dec 25 21:07:11 +0000 2018
Tue Dec 25 19:55:02 +0000 2018
Tue Dec 25 19:53:03 +0000 2018
Tue Dec 25 19:46:50 +0000 2018
Tue Dec 25 18:26:25 +0000 2018
Tue Dec 25 18:18:17 +0000 2018
Tue Dec 25 18:17:22 +0000 2018
Tue Dec 25 17:54:49 +0000 2018
Tue Dec 25 17:41:41 +0000 2018
Tue Dec 25 17:14:23 +0000 2018
Tue Dec 25 17:05:26 +0000 2018
Tue Dec 25 16:05:16 +0000 2018
Tue Dec 25 15:03:45 +0000 2018
Tue Dec 25 13:16:41 +0000 2018
Mon Dec 24 22:34:42 +0000 2018
Mon Dec 24 22:33:54 +0000 2018
Mon Dec 

Thu Nov 22 00:41:53 +0000 2018
Wed Nov 21 23:48:53 +0000 2018
Wed Nov 21 23:12:01 +0000 2018
Wed Nov 21 18:56:42 +0000 2018
Wed Nov 21 17:35:33 +0000 2018
Wed Nov 21 17:35:03 +0000 2018
Wed Nov 21 17:33:22 +0000 2018
Wed Nov 21 16:56:18 +0000 2018
Wed Nov 21 03:15:45 +0000 2018
Tue Nov 20 22:28:49 +0000 2018
Tue Nov 20 20:27:48 +0000 2018
Tue Nov 20 20:22:38 +0000 2018
Tue Nov 20 20:16:05 +0000 2018
Tue Nov 20 20:07:34 +0000 2018
Tue Nov 20 20:01:44 +0000 2018
Tue Nov 20 19:47:23 +0000 2018
Tue Nov 20 19:43:30 +0000 2018
Tue Nov 20 17:34:39 +0000 2018
Mon Nov 19 23:40:17 +0000 2018
Mon Nov 19 20:13:31 +0000 2018
Mon Nov 19 19:07:46 +0000 2018
Mon Nov 19 18:43:35 +0000 2018
Mon Nov 19 18:34:51 +0000 2018
Mon Nov 19 18:30:56 +0000 2018
Mon Nov 19 18:22:51 +0000 2018
Mon Nov 19 18:21:54 +0000 2018
Mon Nov 19 18:14:52 +0000 2018
Mon Nov 19 01:09:24 +0000 2018
Mon Nov 19 01:06:34 +0000 2018
Mon Nov 19 01:01:31 +0000 2018
Sun Nov 18 22:10:52 +0000 2018
Sun Nov 18 00:51:26 +0000 2018
Sun Nov 

Tue Oct 02 17:24:56 +0000 2018
Tue Oct 02 16:21:48 +0000 2018
Tue Oct 02 16:20:05 +0000 2018
Tue Oct 02 16:18:27 +0000 2018
Tue Oct 02 16:17:18 +0000 2018
Tue Oct 02 03:28:19 +0000 2018
Tue Oct 02 00:25:35 +0000 2018
Mon Oct 01 18:48:56 +0000 2018
Mon Oct 01 16:14:47 +0000 2018
Mon Oct 01 16:14:34 +0000 2018
Mon Oct 01 16:13:10 +0000 2018
Mon Oct 01 01:03:13 +0000 2018
Mon Oct 01 00:51:23 +0000 2018
Sat Sep 29 22:36:23 +0000 2018
Sat Sep 29 22:32:30 +0000 2018
Sat Sep 29 20:51:58 +0000 2018
Sat Sep 29 20:40:11 +0000 2018
Sat Sep 29 16:43:58 +0000 2018
Sat Sep 29 04:38:05 +0000 2018
Fri Sep 28 23:10:39 +0000 2018
Fri Sep 28 17:25:22 +0000 2018
Fri Sep 28 00:26:53 +0000 2018
Thu Sep 27 20:05:04 +0000 2018
Thu Sep 27 14:34:38 +0000 2018
Thu Sep 27 00:33:22 +0000 2018
Thu Sep 27 00:14:45 +0000 2018
Thu Sep 27 00:07:58 +0000 2018
Thu Sep 27 00:05:43 +0000 2018
Wed Sep 26 21:18:59 +0000 2018
Wed Sep 26 21:15:07 +0000 2018
Wed Sep 26 21:13:05 +0000 2018
Wed Sep 26 04:17:04 +0000 2018
Wed Sep 

Mon Aug 20 17:07:41 +0000 2018
Mon Aug 20 16:16:25 +0000 2018
Mon Aug 20 16:12:37 +0000 2018
Mon Aug 20 15:54:15 +0000 2018
Mon Aug 20 15:30:32 +0000 2018
Mon Aug 20 15:20:46 +0000 2018
Mon Aug 20 15:00:46 +0000 2018
Mon Aug 20 01:21:38 +0000 2018
Sun Aug 19 23:24:42 +0000 2018
Sun Aug 19 23:23:33 +0000 2018
Sun Aug 19 18:17:32 +0000 2018
Sun Aug 19 01:01:52 +0000 2018
Sat Aug 18 22:21:33 +0000 2018
Sat Aug 18 21:37:16 +0000 2018
Sat Aug 18 21:25:12 +0000 2018
Sat Aug 18 21:07:07 +0000 2018
Sat Aug 18 20:54:17 +0000 2018
Sat Aug 18 20:49:47 +0000 2018
Sat Aug 18 18:46:35 +0000 2018
Sat Aug 18 18:42:08 +0000 2018
Sat Aug 18 17:12:50 +0000 2018
Sat Aug 18 14:54:00 +0000 2018
Sat Aug 18 12:47:11 +0000 2018
Sat Aug 18 00:18:42 +0000 2018
Fri Aug 17 21:29:33 +0000 2018
Fri Aug 17 17:40:35 +0000 2018
Fri Aug 17 16:33:00 +0000 2018
Fri Aug 17 16:31:55 +0000 2018
Fri Aug 17 16:21:40 +0000 2018
Fri Aug 17 16:21:19 +0000 2018
Fri Aug 17 16:14:56 +0000 2018
Fri Aug 17 15:55:57 +0000 2018
Thu Aug 

Sat Aug 11 13:36:34 +0000 2018
Sat Aug 11 13:33:55 +0000 2018
Sat Aug 11 13:32:36 +0000 2018
Sat Aug 11 13:32:29 +0000 2018
Sat Aug 11 13:31:45 +0000 2018
Sat Aug 11 13:31:12 +0000 2018
Sat Aug 11 13:31:11 +0000 2018
Sat Aug 11 13:28:37 +0000 2018
Sat Aug 11 13:28:03 +0000 2018
Sat Aug 11 13:27:27 +0000 2018
Sat Aug 11 13:26:27 +0000 2018
Sat Aug 11 13:25:34 +0000 2018
Sat Aug 11 13:24:44 +0000 2018
Sat Aug 11 13:22:11 +0000 2018
Sat Aug 11 13:20:45 +0000 2018
Sat Aug 11 13:19:35 +0000 2018
Sat Aug 11 13:17:43 +0000 2018
Sat Aug 11 13:17:13 +0000 2018
Sat Aug 11 13:15:25 +0000 2018
Sat Aug 11 13:10:47 +0000 2018
Sat Aug 11 13:10:33 +0000 2018
Sat Aug 11 13:09:37 +0000 2018
Sat Aug 11 13:08:22 +0000 2018
Sat Aug 11 13:05:29 +0000 2018
Sat Aug 11 13:04:21 +0000 2018
Sat Aug 11 13:00:26 +0000 2018
Sat Aug 11 12:59:35 +0000 2018
Sat Aug 11 12:55:55 +0000 2018
Sat Aug 11 12:54:24 +0000 2018
Sat Aug 11 12:51:49 +0000 2018
Sat Aug 11 12:50:49 +0000 2018
Sat Aug 11 12:49:42 +0000 2018
Sat Aug 

Sat Aug 11 05:05:06 +0000 2018
Sat Aug 11 05:03:02 +0000 2018
Sat Aug 11 05:02:06 +0000 2018
Sat Aug 11 05:01:12 +0000 2018
Sat Aug 11 04:57:25 +0000 2018
Sat Aug 11 04:57:18 +0000 2018
Sat Aug 11 04:56:13 +0000 2018
Sat Aug 11 04:56:09 +0000 2018
Sat Aug 11 04:55:35 +0000 2018
Sat Aug 11 04:54:55 +0000 2018
Sat Aug 11 04:54:15 +0000 2018
Sat Aug 11 04:52:36 +0000 2018
Sat Aug 11 04:52:27 +0000 2018
Sat Aug 11 04:52:27 +0000 2018
Sat Aug 11 04:51:57 +0000 2018
Sat Aug 11 04:51:29 +0000 2018
Sat Aug 11 04:51:28 +0000 2018
Sat Aug 11 04:49:16 +0000 2018
Sat Aug 11 04:47:42 +0000 2018
Sat Aug 11 04:46:14 +0000 2018
Sat Aug 11 04:46:00 +0000 2018
Sat Aug 11 04:45:21 +0000 2018
Sat Aug 11 04:43:53 +0000 2018
Sat Aug 11 04:43:32 +0000 2018
Sat Aug 11 04:42:41 +0000 2018
Sat Aug 11 04:42:12 +0000 2018
Sat Aug 11 04:39:48 +0000 2018
Sat Aug 11 04:39:41 +0000 2018
Sat Aug 11 04:37:44 +0000 2018
Sat Aug 11 04:37:30 +0000 2018
Sat Aug 11 04:37:03 +0000 2018
Sat Aug 11 04:32:45 +0000 2018
Sat Aug 

Sat Aug 11 02:10:25 +0000 2018
Sat Aug 11 02:09:13 +0000 2018
Sat Aug 11 02:09:08 +0000 2018
Sat Aug 11 02:08:48 +0000 2018
Sat Aug 11 02:08:16 +0000 2018
Sat Aug 11 02:08:09 +0000 2018
Sat Aug 11 02:07:58 +0000 2018
Sat Aug 11 02:07:48 +0000 2018
Sat Aug 11 02:07:34 +0000 2018
Sat Aug 11 02:07:01 +0000 2018
Sat Aug 11 02:06:34 +0000 2018
Sat Aug 11 02:04:48 +0000 2018
Sat Aug 11 02:04:05 +0000 2018
Sat Aug 11 02:03:23 +0000 2018
Sat Aug 11 02:03:06 +0000 2018
Sat Aug 11 02:02:49 +0000 2018
Sat Aug 11 02:02:44 +0000 2018
Sat Aug 11 02:02:37 +0000 2018
Sat Aug 11 02:02:36 +0000 2018
Sat Aug 11 02:02:25 +0000 2018
Sat Aug 11 02:02:21 +0000 2018
Sat Aug 11 02:02:06 +0000 2018
Sat Aug 11 02:01:04 +0000 2018
Sat Aug 11 02:00:25 +0000 2018
Sat Aug 11 02:00:17 +0000 2018
Sat Aug 11 02:00:17 +0000 2018
Sat Aug 11 02:00:05 +0000 2018
Sat Aug 11 02:00:04 +0000 2018
Sat Aug 11 01:59:24 +0000 2018
Sat Aug 11 01:59:07 +0000 2018
Sat Aug 11 01:58:24 +0000 2018
Sat Aug 11 01:57:38 +0000 2018
Sat Aug 

Fri Aug 10 20:56:36 +0000 2018
Fri Aug 10 20:49:19 +0000 2018
Fri Aug 10 18:43:26 +0000 2018
Fri Aug 10 18:38:22 +0000 2018
Fri Aug 10 18:29:29 +0000 2018
Fri Aug 10 18:12:18 +0000 2018
Fri Aug 10 17:56:00 +0000 2018
Fri Aug 10 17:30:12 +0000 2018
Fri Aug 10 17:19:32 +0000 2018
Fri Aug 10 17:01:01 +0000 2018


Notice, I have used `try` and `except` statements to avoid some "errors" during the loop execution, I know some twitees do not have any picture or any other information and it could led a error, which is not a error, It is just a lack of information that I could assign Null values to theses cases.

In [8]:
# Printing the first 5 rows of twt_json.
pd.DataFrame(twt_json)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,retweet_count,favorite_count,source,full_text,expanded_urls,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp
0,1078007085848240129,Wed Dec 26 19:18:12 +0000 2018,1.078007e+18,3.441648e+08,0,121,"<a href=""http://twitter.com/download/iphone"" r...",@brandimotamedi i hope you love it ☺️,,,,
1,1078007032869998592,Wed Dec 26 19:17:59 +0000 2018,NaN,NaN,192,0,"<a href=""http://twitter.com/download/iphone"" r...","RT @brandimotamedi: Honestly, 13/10 Christmas ...",http://pbs.twimg.com/media/DvXZi5pUwAADyWw.jpg,1078006885935067136,4196983835,Wed Dec 26 19:17:24 +0000 2018
2,1077988498597134343,Wed Dec 26 18:04:20 +0000 2018,NaN,NaN,1298,15486,"<a href=""http://twitter.com/download/iphone"" r...",Thank you for following along with our holiday...,,,,
3,1077979079570272261,Wed Dec 26 17:26:55 +0000 2018,1.077977e+18,7.409112e+17,0,86,"<a href=""http://twitter.com/download/iphone"" r...",@ashley_stasak @Lin_Manuel @jonnysun @TeeRico_...,,,,
4,1077976428187131904,Wed Dec 26 17:16:22 +0000 2018,NaN,7.992370e+07,87,5073,"<a href=""http://twitter.com/download/iphone"" r...",@Lin_Manuel @jonnysun look what i got ☺️ https...,http://pbs.twimg.com/media/DvW915VXgAI7XlO.jpg,,,
5,1077764922698797057,Wed Dec 26 03:15:56 +0000 2018,NaN,NaN,3847,52096,"<a href=""http://twitter.com/download/iphone"" r...",This is Charlie. He was out and about collecti...,http://pbs.twimg.com/media/DvT9eyPWsAApU-T.jpg,,,
6,1077752281779572737,Wed Dec 26 02:25:42 +0000 2018,1.077752e+18,3.830097e+09,2,399,"<a href=""http://twitter.com/download/iphone"" r...",@artdecider oh thank goodness ok i’ll pass tha...,,,,
7,1077751891348676608,Wed Dec 26 02:24:09 +0000 2018,1.077752e+18,3.830097e+09,1,336,"<a href=""http://twitter.com/download/iphone"" r...",@artdecider tbh i get nervous when you don’t c...,,,,
8,1077751671181266944,Wed Dec 26 02:23:16 +0000 2018,NaN,NaN,4485,55662,"<a href=""http://twitter.com/download/iphone"" r...",This is Oliver. He had to shut down his hot co...,http://pbs.twimg.com/media/DvTxbN_W0AcHzBm.jpg,,,
9,1077740758499774464,Wed Dec 26 01:39:54 +0000 2018,1.077740e+18,1.469681e+07,1,503,"<a href=""http://twitter.com/download/iphone"" r...",@PokerVixen no it was the earth,,,,


## 3. Data Assessing <a id='assessing'></a>

This section aims to identify quality and tidyness problems all data sets of this project. For this reason, I will perform a straighforward assessment.

As a part of the grading process, I will divide the Data Assessing into:

* Detect, and;
* Document.

So, for each dataset I will point out all problem I have detected, and later in the Data Cleaning process I will clean and fix these issues.

### 3.1. `.head()`

Let's see the first rows.

In [9]:
df_ach

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [10]:
df_img.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [11]:
len(df_img['tweet_id'])

2075

In [12]:
sum(df_img['tweet_id'].duplicated())

0

In [13]:
len(df_ach['tweet_id'])

2356

In [14]:
sum(df_ach['tweet_id'].duplicated())

0

### Issues

* **Archive Enhanced**
    * 

* **Images** 
    * 

In [15]:
# Creates the user object. The me() method returns the user whose authentication keys were used.
user = api.me()
 
print('Name: ', user.name)
print('Location: ', user.location)
print('Friends: ', str(user.friends_count))

Name:  AH Uyekita
Location:  
Friends:  32


In [16]:
len(twt_json['expanded_urls'])

1970

In [17]:
pd.DataFrame(twt_json).created_at

0       Wed Dec 26 19:18:12 +0000 2018
1       Wed Dec 26 19:17:59 +0000 2018
2       Wed Dec 26 18:04:20 +0000 2018
3       Wed Dec 26 17:26:55 +0000 2018
4       Wed Dec 26 17:16:22 +0000 2018
5       Wed Dec 26 03:15:56 +0000 2018
6       Wed Dec 26 02:25:42 +0000 2018
7       Wed Dec 26 02:24:09 +0000 2018
8       Wed Dec 26 02:23:16 +0000 2018
9       Wed Dec 26 01:39:54 +0000 2018
10      Wed Dec 26 01:27:12 +0000 2018
11      Wed Dec 26 00:40:00 +0000 2018
12      Wed Dec 26 00:35:08 +0000 2018
13      Wed Dec 26 00:23:18 +0000 2018
14      Tue Dec 25 23:15:32 +0000 2018
15      Tue Dec 25 22:27:00 +0000 2018
16      Tue Dec 25 21:07:11 +0000 2018
17      Tue Dec 25 19:55:02 +0000 2018
18      Tue Dec 25 19:53:03 +0000 2018
19      Tue Dec 25 19:46:50 +0000 2018
20      Tue Dec 25 18:26:25 +0000 2018
21      Tue Dec 25 18:18:17 +0000 2018
22      Tue Dec 25 18:17:22 +0000 2018
23      Tue Dec 25 17:54:49 +0000 2018
24      Tue Dec 25 17:41:41 +0000 2018
25      Tue Dec 25 17:14:

## Y. Appendix <a id='appendix'></a>

### A1. JSON file

I have indented this fragment of JSON file, just to understand what is the components, and later help me to subset/indexing the needs.

See more in [Twitter Dev Website][twt_dev_wb].

[twt_dev_wb]: https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline

```json
Status(_api=<tweepy.api.API object at 0x0B288EF0>,
_json={'created_at': 'Wed Dec 26 19:18:12 +0000 2018',
       'id': 1078007085848240129,
       'id_str': '1078007085848240129',
       'text': '@brandimotamedi i hope you love it ☺️',
       'truncated': False,
       'entities': {'hashtags': [],
                    'symbols': [],
                    'user_mentions': [{'screen_name':
                                       'brandimotamedi',
                                       'name': 'Brandi Motamedi',
                                       'id': 344164849,
                                       'id_str': '344164849',
                                       'indices': [0, 15]}],
                    'urls': []},
       'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       'in_reply_to_status_id': 1078006885935067136,
       'in_reply_to_status_id_str': '1078006885935067136',
       'in_reply_to_user_id': 344164849,
       'in_reply_to_user_id_str': '344164849',
       'in_reply_to_screen_name': 'brandimotamedi',
       'user': {'id': 4196983835,
                'id_str': '4196983835',
                'name': 'WeRateDogs™',
                'screen_name': 'dog_rates',
                'location': 'merch  ⇨',
                'description': 'Your Only Source For Professional Dog Ratings ⠀ ⠀IG, FB, Snapchat ⇨ WeRateDogs ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ Business: dogratingtwitter@gmail.com',
                'url': 'https://t.co/N7sNNHAEXS',
                'entities': {'url': {'urls': [{'url': 'https://t.co/N7sNNHAEXS',
                                                      'expanded_url': 'http://weratedogs.com',
                                                      'display_url': 'weratedogs.com',
                                                      'indices': [0, 23]}]},
                                    'description': {'urls': []}},
                'protected': False,
                'followers_count': 7549717,
                'friends_count': 12,
                'listed_count': 5661,
                'created_at': 'Sun Nov 15 21:41:29 +0000 2015',
                'favourites_count': 140650,
                'utc_offset': None,
                'time_zone': None,
                'geo_enabled': True,
                'verified': True,
                'statuses_count': 9467,
                'lang': 'en',
                'contributors_enabled': False,
                'is_translator': False,
                'is_translation_enabled': False,
                'profile_background_color': '000000',
                'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
                'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
                'profile_background_tile': False,
                'profile_image_url': 'http://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
                'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
                'profile_banner_url': 'https://pbs.twimg.com/profile_banners/4196983835/1544368760',
                'profile_link_color': 'F5ABB5',
                'profile_sidebar_border_color': '000000',
                'profile_sidebar_fill_color': '000000',
                'profile_text_color': '000000',
                'profile_use_background_image': False,
                'has_extended_profile': False,
                'default_profile': False,
                'default_profile_image': False,
                'following': False,
                'follow_request_sent': False,
                'notifications': False,
                'translator_type': 'none'},
       'geo': None,
       'coordinates': None,
       'place': None,
       'contributors': None,
       'is_quote_status': False,
       'retweet_count': 0,
       'favorite_count': 115,
       'favorited': False,
       'retweeted': False,
       'lang': 'en'},
created_at=datetime.datetime(2018, 12, 26, 19, 18, 12),
id=1078007085848240129,
id_str='1078007085848240129',
text='@brandimotamedi i hope you love it ☺️',
truncated=False,
entities={'hashtags': [],
          'symbols': [],
          'user_mentions': [{'screen_name': 'brandimotamedi',
                             'name': 'Brandi Motamedi',
                             'id': 344164849,
                             'id_str': '344164849',
                             'indices': [0, 15]}],
          'urls': []},
source='Twitter for iPhone',
source_url='http://twitter.com/download/iphone',
in_reply_to_status_id=1078006885935067136,
in_reply_to_status_id_str='1078006885935067136',
in_reply_to_user_id=344164849,
in_reply_to_user_id_str='344164849',
in_reply_to_screen_name='brandimotamedi',
author=User(_api=<tweepy.api.API object at 0x0B288EF0>,
            _json={'id': 4196983835,
                   'id_str': '4196983835',
                   'name': 'WeRateDogs™',
                   'screen_name': 'dog_rates',
                   'location': 'merch  ⇨', 'description': 'Your Only Source For Professional Dog Ratings ⠀ ⠀IG, FB, Snapchat ⇨ WeRateDogs ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ Business: dogratingtwitter@gmail.com',
                   'url': 'https://t.co/N7sNNHAEXS',
                   'entities': {'url': {'urls': [{'url': 'https://t.co/N7sNNHAEXS',
                                                         'expanded_url': 'http://weratedogs.com',
                                                         'display_url': 'weratedogs.com',
                                                         'indices': [0, 23]}]},
                                       'description': {'urls': []}},
                   'protected': False,
                   'followers_count': 7549717,
                   'friends_count': 12,
                   'listed_count': 5661,
                   'created_at': 'Sun Nov 15 21:41:29 +0000 2015',
                   'favourites_count': 140650,
                   'utc_offset': None,
                   'time_zone': None,
                   'geo_enabled': True,
                   'verified': True,
                   'statuses_count': 9467,
                   'lang': 'en',
                   'contributors_enabled': False,
                   'is_translator': False,
                   'is_translation_enabled': False,
                   'profile_background_color': '000000',
                   'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
                   'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
                   'profile_background_tile': False,
                   'profile_image_url': 'http://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
                   'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
                   'profile_banner_url': 'https://pbs.twimg.com/profile_banners/4196983835/1544368760',
                   'profile_link_color': 'F5ABB5',
                   'profile_sidebar_border_color': '000000',
                   'profile_sidebar_fill_color': '000000',
                   'profile_text_color': '000000',
                   'profile_use_background_image': False,
                   'has_extended_profile': False,
                   'default_profile': False,
                   'default_profile_image': False,
                   'following': False,
                   'follow_request_sent': False,
                   'notifications': False,
                   'translator_type': 'none'},
            id=4196983835,
            id_str='4196983835',
            name='WeRateDogs™',
            screen_name='dog_rates',
            location='merch  ⇨',
            description='Your Only Source For Professional Dog Ratings ⠀ ⠀IG, FB, Snapchat ⇨ WeRateDogs ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ Business: dogratingtwitter@gmail.com',
            url='https://t.co/N7sNNHAEXS',
            entities={'url': {'urls': [{'url': 'https://t.co/N7sNNHAEXS',
                                               'expanded_url': 'http://weratedogs.com',
                                               'display_url': 'weratedogs.com',
                                               'indices': [0, 23]}]},
                             'description': {'urls': []}},
            protected=False,
            followers_count=7549717,
            friends_count=12,
            listed_count=5661,
            created_at=datetime.datetime(2015, 11, 15, 21, 41, 29),
            favourites_count=140650,
            utc_offset=None,
            time_zone=None,
            geo_enabled=True,
            verified=True,
            statuses_count=9467,
            lang='en',
            contributors_enabled=False,
            is_translator=False,
            is_translation_enabled=False,
            profile_background_color='000000',
            profile_background_image_url='http://abs.twimg.com/images/themes/theme1/bg.png',
            profile_background_image_url_https='https://abs.twimg.com/images/themes/theme1/bg.png',
            profile_background_tile=False,
            profile_image_url='http://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
            profile_image_url_https='https://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
            profile_banner_url='https://pbs.twimg.com/profile_banners/4196983835/1544368760',
            profile_link_color='F5ABB5',
            profile_sidebar_border_color='000000',
            profile_sidebar_fill_color='000000',
            profile_text_color='000000',
            profile_use_background_image=False,
            has_extended_profile=False,
            default_profile=False,
            default_profile_image=False,
            following=False,
            follow_request_sent=False,
            notifications=False, translator_type='none'),
user=User(_api=<tweepy.api.API object at 0x0B288EF0>,
          _json={'id': 4196983835,
                 'id_str': '4196983835',
                 'name': 'WeRateDogs™',
                 'screen_name': 'dog_rates',
                 'location': 'merch  ⇨',
                 'description': 'Your Only Source For Professional Dog Ratings ⠀ ⠀IG, FB, Snapchat ⇨ WeRateDogs ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ Business: dogratingtwitter@gmail.com',
                 'url': 'https://t.co/N7sNNHAEXS',
                 'entities': {'url': {'urls': [{'url': 'https://t.co/N7sNNHAEXS',
                                                'expanded_url': 'http://weratedogs.com',
                                                'display_url': 'weratedogs.com',
                                                'indices': [0, 23]}]},
                                     'description': {'urls': []}},
                             'protected': False,
                             'followers_count': 7549717,
                             'friends_count': 12,
                             'listed_count': 5661,
                             'created_at': 'Sun Nov 15 21:41:29 +0000 2015',
                             'favourites_count': 140650,
                             'utc_offset': None,
                             'time_zone': None,
                             'geo_enabled': True,
                             'verified': True,
                             'statuses_count': 9467,
                             'lang': 'en',
                             'contributors_enabled': False,
                             'is_translator': False,
                             'is_translation_enabled': False,
                             'profile_background_color': '000000',
                             'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
                             'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
                             'profile_background_tile': False,
                             'profile_image_url': 'http://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
                             'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
                             'profile_banner_url': 'https://pbs.twimg.com/profile_banners/4196983835/1544368760',
                             'profile_link_color': 'F5ABB5',
                             'profile_sidebar_border_color': '000000',
                             'profile_sidebar_fill_color': '000000',
                             'profile_text_color': '000000',
                             'profile_use_background_image': False,
                             'has_extended_profile': False,
                             'default_profile': False,
                             'default_profile_image': False,
                             'following': False,
                             'follow_request_sent': False,
                             'notifications': False,
                             'translator_type': 'none'},
          id=4196983835,
          id_str='4196983835',
          name='WeRateDogs™',
          screen_name='dog_rates',
          location='merch  ⇨',
          description='Your Only Source For Professional Dog Ratings ⠀ ⠀IG, FB, Snapchat ⇨ WeRateDogs ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ Business: dogratingtwitter@gmail.com',
          url='https://t.co/N7sNNHAEXS',
          entities={'url': {'urls': [{'url': 'https://t.co/N7sNNHAEXS',
                                             'expanded_url': 'http://weratedogs.com',
                                             'display_url': 'weratedogs.com',
                                             'indices': [0, 23]}]},
                            'description': {'urls': []}},
          protected=False,
          followers_count=7549717,
          friends_count=12,
          listed_count=5661,
          created_at=datetime.datetime(2015, 11, 15, 21, 41, 29),
          favourites_count=140650,
          utc_offset=None,
          time_zone=None,
          geo_enabled=True,
          verified=True,
          statuses_count=9467,
          lang='en',
          contributors_enabled=False,
          is_translator=False,
          is_translation_enabled=False,
          profile_background_color='000000',
          profile_background_image_url='http://abs.twimg.com/images/themes/theme1/bg.png',
          profile_background_image_url_https='https://abs.twimg.com/images/themes/theme1/bg.png',
          profile_background_tile=False,
          profile_image_url='http://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
          profile_image_url_https='https://pbs.twimg.com/profile_images/1072659905235042304/nm3HWlPG_normal.jpg',
          profile_banner_url='https://pbs.twimg.com/profile_banners/4196983835/1544368760',
          profile_link_color='F5ABB5',
          profile_sidebar_border_color='000000',
          profile_sidebar_fill_color='000000',
          profile_text_color='000000',
          profile_use_background_image=False,
          has_extended_profile=False,
          default_profile=False,
          default_profile_image=False,
          following=False,
          follow_request_sent=False,
          notifications=False,
          translator_type='none'),
geo=None,
coordinates=None,
place=None,
contributors=None,
is_quote_status=False,
retweet_count=0,
favorite_count=115,
favorited=False,
retweeted=False,
lang='en')
```
***

## X. References <a id='references'></a>

I have consulted this websites to perform this project.

* Roesslein, J.; tweepy, Version 3.7.0, [website](https://tweepy.readthedocs.io/en/3.7.0/index.html);
